In [13]:
# !pip install pyarrow

In [14]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

#import xgbtune 

In [15]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,...,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year,short_ema_50,long_ema_200,macd_50_200,macd_hist_50_200_9
0,2004-09-07,0.632143,0.646250,0.629107,0.638571,0.539257,301957600,1,NaN,NaN,...,NaN,1,7,9,2004,37,0.638571,0.638571,0.000000,0.000000
1,2004-09-08,0.637500,0.653036,0.637143,0.649107,0.548155,343526400,0,NaN,NaN,...,NaN,2,8,9,2004,37,0.638984,0.638676,0.000308,0.000247
2,2004-09-09,0.644643,0.648214,0.630000,0.637500,0.538353,461339200,1,NaN,NaN,...,NaN,3,9,9,2004,37,0.638926,0.638664,0.000262,0.000160
3,2004-09-10,0.636786,0.646964,0.633214,0.640536,0.540917,328014400,0,NaN,NaN,...,NaN,4,10,9,2004,37,0.638989,0.638683,0.000306,0.000164
4,2004-09-13,0.640714,0.644107,0.630714,0.635536,0.536694,281976800,0,NaN,NaN,...,NaN,0,13,9,2004,38,0.638854,0.638651,0.000202,0.000048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,2024-07-17,229.449997,231.460007,226.639999,228.880005,228.615387,57345900,0,3.289316,0.017871,...,0.005091,2,17,7,2024,29,207.897518,188.908818,18.988700,2.418547
4999,2024-07-18,230.279999,230.440002,222.270004,224.179993,223.920807,66034600,1,3.006525,0.017869,...,0.006392,3,18,7,2024,29,208.536046,189.259775,19.276271,2.164895
5000,2024-07-19,224.820007,226.800003,223.279999,224.309998,224.050659,49151500,0,3.906402,0.020045,...,0.012702,4,19,7,2024,29,209.154632,189.608533,19.546099,1.947778
5001,2024-07-22,227.009995,227.779999,223.089996,223.960007,223.701080,48201800,1,4.158552,0.020108,...,0.005147,0,22,7,2024,30,209.735235,189.950339,19.784897,1.749260


In [16]:
# target = "target_close"
target = "close_percent_change_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,close_percent_change_1
date,
2004-09-07,0.016499
2004-09-08,-0.017881
2004-09-09,0.004762
2004-09-10,-0.007806
2004-09-13,-0.002810
...,...
2024-07-17,-0.020535
2024-07-18,0.000580
2024-07-19,-0.001560


In [17]:
x

,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year,short_ema_50,long_ema_200,macd_50_200,macd_hist_50_200_9
date,,,,,,,,,,,,,,,,,,,,,
2004-09-07,0.632143,0.646250,0.629107,0.638571,0.539257,301957600,1,NaN,NaN,0.017143,...,NaN,1,7,9,2004,37,0.638571,0.638571,0.000000,0.000000
2004-09-08,0.637500,0.653036,0.637143,0.649107,0.548155,343526400,0,NaN,NaN,0.015893,...,NaN,2,8,9,2004,37,0.638984,0.638676,0.000308,0.000247
2004-09-09,0.644643,0.648214,0.630000,0.637500,0.538353,461339200,1,NaN,NaN,0.018214,...,NaN,3,9,9,2004,37,0.638926,0.638664,0.000262,0.000160
2004-09-10,0.636786,0.646964,0.633214,0.640536,0.540917,328014400,0,NaN,NaN,0.013750,...,NaN,4,10,9,2004,37,0.638989,0.638683,0.000306,0.000164
2004-09-13,0.640714,0.644107,0.630714,0.635536,0.536694,281976800,0,NaN,NaN,0.013393,...,NaN,0,13,9,2004,38,0.638854,0.638651,0.000202,0.000048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17,229.449997,231.460007,226.639999,228.880005,228.615387,57345900,0,3.289316,0.017871,4.820007,...,0.005091,2,17,7,2024,29,207.897518,188.908818,18.988700,2.418547
2024-07-18,230.279999,230.440002,222.270004,224.179993,223.920807,66034600,1,3.006525,0.017869,8.169998,...,0.006392,3,18,7,2024,29,208.536046,189.259775,19.276271,2.164895
2024-07-19,224.820007,226.800003,223.279999,224.309998,224.050659,49151500,0,3.906402,0.020045,3.520004,...,0.012702,4,19,7,2024,29,209.154632,189.608533,19.546099,1.947778


In [18]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,crossed_close_bollinger_lower_30_close_event_name,object
0,crossed_close_close_bollinger_upper_30_event_name,object
0,dividends_event_name,object
0,splits_event_name,object


In [19]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64 (XGBoosted models cannot use strings, but categories as enumerated values)
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: crossed_close_bollinger_lower_30_close_event_name
Dropped Column: crossed_close_close_bollinger_upper_30_event_name
Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [20]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [21]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [22]:
# y_test

In [23]:
x_train

,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,high_low_7,...,close_percent_change_1_lag_60,day_of_week,day_of_month,month,year,week_of_year,short_ema_50,long_ema_200,macd_50_200,macd_hist_50_200_9
date,,,,,,,,,,,,,,,,,,,,,
2006-11-21,3.122143,3.164286,3.111071,3.164286,2.672160,622666800,1,0.044099,0.010560,0.053215,...,-0.007465,1,21,11,2006,47,2.805814,2.466780,0.339033,0.022352
2004-11-08,0.969107,0.990179,0.961786,0.971071,0.820045,526920800,0,0.023534,0.019280,0.028393,...,NaN,0,8,11,2004,46,0.790140,0.688567,0.101573,0.019495
2019-10-31,61.810001,62.292500,59.314999,62.189999,60.177780,139162000,1,0.623345,0.011459,2.977501,...,0.022056,3,31,10,2019,44,56.806612,51.497404,5.309208,0.405638
2005-03-15,1.451429,1.469286,1.437500,1.462857,1.235347,508608800,1,0.042763,0.023742,0.031786,...,-0.024174,1,15,3,2005,11,1.394168,1.051170,0.342998,-0.003151
2006-08-25,2.405000,2.466071,2.403929,2.455357,2.073488,543958800,0,0.017307,0.011383,0.062142,...,-0.008203,4,25,8,2006,34,2.273914,2.208415,0.065499,0.017409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-15,118.720001,121.199997,118.150002,120.709999,117.963219,112559200,0,3.755824,0.029896,3.049995,...,-0.045516,3,15,10,2020,42,113.353504,92.621869,20.731635,-0.029506
2011-04-25,12.512143,12.633929,12.510714,12.607500,10.646721,266546000,0,0.373115,0.022021,0.123216,...,-0.020716,0,25,4,2011,17,12.195034,11.145712,1.049322,-0.041468
2009-03-30,3.732500,3.750357,3.664643,3.731786,3.151401,502796000,1,0.090170,0.020545,0.085714,...,0.063269,0,30,3,2009,14,3.441679,4.017440,-0.575760,0.067659


In [24]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [25]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.05, subsample=0.9, colsample_bytree=0.4)
model.fit(x_train, y_train)

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

tune_params = {
    # "n_estimators": [100, 150],
    # "max_depth": [5, 6,],
    # "learning_rate": [0.05, 0.1],

    "n_estimators": [400, 450, 500, 550, 600],
    "max_depth": [6, 7, 8, 9, 11],
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4,],
    "subsample": [0.7, 0.8, 0.9, 1.0, 1.1],
    "colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5],

}

# {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.3}
# {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.2}

# xgb_model = xgb.XGBRegressor()
# # model = GridSearchCV(estimator=xgb_model, param_grid=tune_params)
# model = HalvingRandomSearchCV(estimator=xgb_model, param_distributions=tune_params)
# model.fit(x_train, y_train)

# print(model.best_params_)

pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [26]:
# print(model.best_params_)
# model


In [27]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

,close_percent_change_1,predicted
date,,
2008-06-02,-0.003923,-0.017734
2015-05-13,0.023331,0.009800
2006-03-29,0.006738,0.037001
2017-02-16,0.002734,0.002597
2020-11-13,0.008720,0.012945
...,...,...
2011-12-12,-0.007733,-0.008307
2015-08-10,-0.052038,-0.011233
2005-05-10,-0.022240,-0.015238


In [28]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

np.float64(0.013311032404412294)

In [29]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

np.float64(0.00943012766798978)

In [30]:
# 0.01455269590461311
# 0.013673832772506729
# 0.012685448862479572

In [31]:
# rmsle
rmsle = np.sqrt(np.mean(np.log1p(np.abs(y_test["predicted"] - y_test[target])) ** 2))
rmsle

np.float64(0.013107173572420704)

In [32]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

np.float64(inf)

In [33]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")